In [40]:
import tensorflow as tf
import tkinter as tk
from tkinter import *
import numpy as np
import win32gui
from PIL import Image, ImageGrab
from keras.models import load_model
from tensorflow.keras import datasets, layers, models
import keras
from keras import utils as np_utils

In [42]:
"""train model with Mnist data"""
(train_x, train_y), (test_x, test_y) = datasets.mnist.load_data()

print(train_x.shape, train_y.shape)

train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# convert class vectors to binary class matrices
train_y = keras.utils.np_utils.to_categorical(train_y, 10)
test_y = keras.utils.np_utils.to_categorical(test_y, 10)

train_x = train_x.astype('float32')
test_x = test_x.astype('float32')
train_x /= 255
test_x /= 255
print('train_x shape:', train_x.shape)
print(train_x.shape[0], 'train samples')
print(test_x.shape[0], 'test samples')

batch_size = 128
num_classes = 10
epochs = 10

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(5, 5),activation='relu',input_shape=input_shape))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])

hist = model.fit(train_x, train_y,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(test_x, test_y))
print("The model has successfully trained")

score = model.evaluate(test_x, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save('mnist.h5')
print("Saving the model as mnist.h5")

(60000, 28, 28) (60000,)
train_x shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/10
469/469 [==============================] - 8s 5ms/step - loss: 0.3917 - accuracy: 0.8813 - val_loss: 0.0619 - val_accuracy: 0.9820
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.1139 - accuracy: 0.9704 - val_loss: 0.0497 - val_accuracy: 0.9835
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0796 - accuracy: 0.9796 - val_loss: 0.0358 - val_accuracy: 0.9890
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0648 - accuracy: 0.9838 - val_loss: 0.0366 - val_accuracy: 0.9896
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0560 - accuracy: 0.9856 - val_loss: 0.0261 - val_accuracy: 0.9916
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0491 - accuracy: 0.9874 - val_loss: 0.0361 - val_accuracy: 0.9907
Epoch 7/10
469/469 [==============================

In [58]:
model = load_model('mnist.h5')

"""define function to predict numbers"""
def predict_number(img):
    img = img.resize((28,28)) #resize mnist data to 28x28
    img = img.convert('L') #rgb to gray
    img = np.array(img)
    img = img.reshape(1,28,28,1) #reshape to 28x28, 1 channel
    img = img/255.0 #normalize

    # predict
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

In [59]:
"""Gui app class"""
class gui_app(tk.Tk):
    # app setup
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        self.canvas = tk.Canvas(self,width=300, height=300, bg='white', cursor='cross')
        self.label = tk.Label(self, text='Processing . . .', font=('Calibri', 48))
        self.classify_btn = tk.Button(self, text='Recognize', command= self.classify_handwriting)
        self.button_clear = tk.Button(self, text='Clear', command=self.clear_all)
        
        self.canvas.grid(row=0, column=0, pady=2, sticky=W)
        self.label.grid(row=0, column=0, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        self.canvas.bind('<B1-Motion>',self.draw_lines)
    # clear all fn
    def clear_all(self):
        self.canvas.delete('all')

    # define hand wiriting classification fn
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()
        rect = win32gui.GetWindowRect(HWND)
        x1, y1, x2, y2 = rect
        # print(x1,x2, y1,y2)
        im = ImageGrab.grab((x1+40, y1+40, x2+100, y2+100))
        digit, acc = predict_number(im)
        print(digit)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')
    
    # draw lines fn
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval(self.x-r, self.y-r, self.x+r, self.y+r, fill='black')

    

In [60]:
"""Launch app"""
app = gui_app()
mainloop()

2
